In [79]:
import cv2
import numpy as np

In [80]:
img=cv2.imread("coins.jpeg")

if img is None:
    print("Error: Image not found or unable to load")

max_size = max(img.shape[:2]) 
scale = 700 / max_size if max_size > 700 else 1
img_resized = cv2.resize(img, (0, 0), fx=scale, fy=scale)

In [81]:
gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
binary_img = cv2.adaptiveThreshold(
    blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
)

kernel = np.ones((3, 3), np.uint8)
closed = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernel, iterations=2)

cv2.imshow("Edges", binary_img)
cv2.waitKey(0)  # Wait for a key press
cv2.destroyAllWindows() 

In [82]:
detected_objects, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = img_resized.copy()

# cv2.drawContours(contours, detected_objects, -1, (0, 0, 255), 2)
# cv2.imshow("Contours",contours)
# cv2.waitKey(0)  # Wait for a key press
# cv2.destroyAllWindows() 

min_area = 500 * (scale ** 2)
print(min_area)
circular_shapes = []
for obj in detected_objects:
    perimeter = cv2.arcLength(obj, True)
    area = cv2.contourArea(obj)
    if perimeter > 0:
        circularity = 4 * np.pi * (area / (perimeter ** 2))
        if circularity < 1.2 and area > min_area:
            circular_shapes.append(obj)
        else:
            print(circularity,area)

# print(circular_shapes)

14.717896962706652
0.5390120920947209 0.5
0.5390120952601716 1.0
0.41923162718478296 3.5
0.5715045502581996 2.5
0.34759871469583076 12.0
0.3252234356020246 10.0
0.11227055861809261 5.0
0.12092828481264367 3.5
0.11430091005163992 0.5
0.5449705313375665 6.5
0.5390120952601716 1.0
0.0 0.0
0.17914334800842904 10.0
0.6430287075658971 1.5
0.2734046554276676 5.5
0.6430287075658971 1.5
0.0 0.0
0.48368704557039216 1.5
0.0 0.0
0.0 0.0
0.3665282226244102 8.5
0.6108759556927862 8.0
0.0 0.0
0.21434290252196572 8.0
0.26950604604736045 1.0
0.0 0.0
0.0 0.0
0.46112891113178694 5.5
0.5390120920947209 0.5
0.8001063703614794 3.5
0.21434290252196572 2.0
0.5643015462265515 14.0
0.0 0.0
0.21434290252196572 0.5
0.13763803409759215 14.5
0.2982596215990625 12.0
0.6108759556927862 8.0
0.5390120920947209 0.5
0.5687171522971307 14.5
0.6380493753606354 4.5
0.09247985475540646 0.5
0.6981317007977318 2.0
0.46461165033847796 7.5
0.6430287075658971 1.5
0.0 0.0
0.5715045502581996 2.5
0.0 0.0
0.3042877806238066 5.5
0.0 0

In [83]:
result_img = img_resized.copy()
    
cv2.drawContours(result_img, circular_shapes, -1, (0, 0, 255), 2)
save_path="outline/outline.jpeg"
cv2.imwrite(save_path, result_img)

result_img = img_resized.copy()
mask_layer = np.zeros(result_img.shape[:2], dtype=np.uint8)
mask_3ch = cv2.cvtColor(mask_layer, cv2.COLOR_GRAY2BGR)
masked_result = cv2.bitwise_and(img_resized, mask_3ch)
cv2.drawContours(mask_layer, circular_shapes, -1, 255, thickness=cv2.FILLED)
result_img = cv2.bitwise_and(result_img, result_img, mask=mask_layer)

save_path="contours/contours.jpeg"
cv2.imwrite(save_path, result_img)

True